Test pour le fichier

In [ ]:
import pandas as pd
import requests
import datetime
import boto3

# Paramètres
api_url = "https://api.worldbank.org/v2/fr/country/all/indicator/NY.GDP.MKTP.KD.ZG?format=json&date=2010%3A2020"
aws_access_key = "VOTRE_CLE_D'ACCES_AWS"
aws_secret_key = "VOTRE_CLE_SECRETE_AWS"
s3_bucket_name = "NOM_DE_VOTRE_BUCKET"

# Extraction et transformation des données
def extract_transform_data(api_url):
    response = requests.get(api_url)
    data = response.json()[1]

    data_list = []
    for entry in data:
        indicator = entry["indicator"]["value"]
        country = entry["country"]["value"]
        date = entry["date"]
        value = entry["value"]
        data_list.append({"Indicator": indicator, "Country": country, "Date": date, "Value": value})

    df = pd.DataFrame(data_list)

    # Convertir la colonne "Date" en format datetime
    df["Date"] = pd.to_datetime(df["Date"])

    # Ajouter une colonne pour l'année
    df["Year"] = df["Date"].dt.year

    # Filtrer les données pour les années 2010 à 2020
    df = df[(df["Year"] >= 2010) & (df["Year"] <= 2020)]

    # Pivotez les données pour avoir les années en colonnes
    df_pivot = df.pivot(index="Country", columns="Year", values="Value")
    
    # Remplacer les valeurs NaN par 0
    df_pivot.fillna(0, inplace=True)
    
    # Calculer la moyenne de croissance sur la période 2010-2020
    df_pivot["Average_Growth"] = df_pivot.iloc[:, :-1].mean(axis=1)
    
    # Obtenir les pays avec la plus haute moyenne de croissance
    top_countries = df_pivot.nlargest(10, "Average_Growth")
    
    return df_pivot, top_countries

# Chargement des données dans AWS S3
def load_to_s3(df, s3_bucket_name):
    today = datetime.datetime.today().strftime('%Y_%m_%d')
    file_name = f"sujet_{today}.csv"
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)
    csv_buffer = df.to_csv(index=True)

    s3.upload_fileobj(pd.compat.StringIO(csv_buffer), s3_bucket_name, file_name)
    print(f"Data saved as {file_name} in {s3_bucket_name}")

if _name_ == "_main_":
    data_df, top_countries_df = extract_transform_data(api_url)
    load_to_s3(data_df, s3_bucket_name)
    load_to_s3(top_countries_df, s3_bucket_name)